In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/transformer-drg-style-transfer-master

/content/drive/.shortcut-targets-by-id/1gvgdEyQQFFN43xnL2_DdI4rUtMI5gnmU/transformer-drg-style-transfer-master


In [ ]:
! pip install boto3
! pip install SentencePiece==0.1.94
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 13.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.7 MB/s eta 0:00:00


In [ ]:
import csv
import logging
import os
import random
import sys

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration

In [ ]:
# file paths
data_dir = "/content/drive/MyDrive/transformer-drg-style-transfer-master/data/"
dataset = "yelp" # amazon / yelp / imagecaption
train_0 = os.path.join(data_dir ,"{}/sentiment.train.0".format(dataset))
train_1 = os.path.join(data_dir,"{}/sentiment.train.1".format(dataset))
test_0 = os.path.join(data_dir,"{}/sentiment.test.0".format(dataset))
test_1 = os.path.join(data_dir,"{}/sentiment.test.1".format(dataset))
dev_0 = os.path.join(data_dir,"{}/sentiment.dev.0".format(dataset))
dev_1 = os.path.join(data_dir,"{}/sentiment.dev.1".format(dataset))
reference_0 = os.path.join(data_dir,"{}/reference.0".format(dataset))
reference_1 = os.path.join(data_dir,"{}/reference.1".format(dataset))

In [ ]:
# file paths
data_dir = "/content/drive/MyDrive/transformer-drg-style-transfer-master/data/"
dataset = "yelp" # amazon / yelp / imagecaption
train_0_out = os.path.join(data_dir ,"{}/processed_files_with_t5_with_best_head/sentiment.train.0".format(dataset))
train_1_out = os.path.join(data_dir,"{}/processed_files_with_t5_with_best_head/sentiment.train.1".format(dataset))
test_0_out = os.path.join(data_dir,"{}/processed_files_with_t5_with_best_head/sentiment.test.0".format(dataset))
test_1_out = os.path.join(data_dir,"{}/processed_files_with_t5_with_best_head/sentiment.test.1".format(dataset))
dev_0_out = os.path.join(data_dir,"{}/processed_files_with_t5_with_best_head/sentiment.dev.0".format(dataset))
dev_1_out = os.path.join(data_dir,"{}/processed_files_with_t5_with_best_head/sentiment.dev.1".format(dataset))
reference_0_out = os.path.join(data_dir,"{}/processed_files_with_t5_with_best_head/reference.0".format(dataset))
reference_1_out = os.path.join(data_dir,"{}/processed_files_with_t5_with_best_head/reference.1".format(dataset))

In [21]:
reference_1_out

'/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/reference.1'

In [ ]:
## Model for performing Classification
tokenizer = T5Tokenizer.from_pretrained('t5-base', max_length=70)
model =  T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_base_yelp_sentiment/', return_dict = True)
model.load_state_dict(torch.load('/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_base_yelp_sentiment/pytorch_model.bin', map_location=torch.device(device)))
model.eval()
model.to(device)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
# tokenizer = T5Tokenizer.from_pretrained('t5-base', max_length=70)
# model =  T5ForConditionalGeneration.from_pretrained('t5-base', return_dict = True)
# #model_cls.load_state_dict(torch.load('/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_base_yelp_sentiment/pytorch_model.bin', map_location=torch.device(device)))
# model.eval()

In [ ]:
common_words=['is','are','was','were','has','have','had','a','an','the','this','that','these','those','there','how','i','we',
             'he','she','it','they','them','their','his','him','her','us','our', 'and','in','my','your','you', 'will', 'shall']
common_words_tokens = tokenizer.convert_tokens_to_ids(common_words)
not_to_remove_ids = tokenizer.convert_tokens_to_ids(['</s>', ".", "?", "!"])
not_to_remove_ids += common_words_tokens

In [ ]:
def create_output_file(original_sentences,processed_sentences, output_file, sentiment="<POS>"):
    print(output_file)
    with open(output_file,"w") as fp:
        for sen1,sen2 in zip(original_sentences,processed_sentences):
            if sen1 != None and sen2 != None:
                str1 = sentiment + " <CON_START> " + sen2 + " <START> " + sen1 + " <END>\n"
                fp.write(str1)

In [ ]:
def create_ref_output_file(processed_sentences, output_file,sentiment="<POS>"):
    with open(output_file,"w") as fp:
        for sen in tqdm(processed_sentences):
            if sen != None:
                str1 = sentiment + " <CON_START> " + sen + " <START>\n"
                fp.write(str1)

In [ ]:
def concate_files(inp_files, out_files):
    with open(out_files,"w") as fp:
        for file in inp_files:
            with open(file) as f:
                for line in f:
                    fp.write(line)

In [ ]:
import os
import psutil

def clear_ram_cache():
    """
    Clears the RAM cache on a Linux-based system.
    """
    os.system("sync; echo 1 > /proc/sys/vm/drop_caches")
    #print("RAM cache cleared.")

clear_ram_cache()

In [ ]:
def read_file(file_path):
    with open(file_path) as fp:
        data = fp.read().splitlines()
    return data

In [ ]:
train_0_data = read_file(train_0)
train_1_data = read_file(train_1)
dev_0_data = read_file(dev_0)
dev_1_data = read_file(dev_1)
test_0_data = read_file(test_0)
test_1_data = read_file(test_1)
ref_0_data = read_file(reference_0)
ref_1_data = read_file(reference_1)

In [ ]:
max_seq_len = 72
def run_attn_examples(input_sentences, layer, head, batch_size=128):
    """
    Returns Attention weights for selected Layer and Head along with ids and tokens
    of the input_sentence
    """
    dataset = tokenizer(input_sentences, max_length=72, pad_to_max_length=True, 
                                                          return_tensors="pt", truncation = True)
    dataset = dataset['input_ids']
    
    
    model.to(device)
    attention_weights = [None for z in input_sentences]
    idx = 0
    tokens_to_decode = [None for k in range(len(input_sentences))]
    ids_to_decode = [None for k in range(len(input_sentences))]
    for j,sen in enumerate(tqdm(input_sentences)):

      text_tokens = tokenizer.tokenize(sen)
      if len(text_tokens) >= max_seq_len-2:
          text_tokens = text_tokens[:max_seq_len-4]
      tokens =  text_tokens + ["</s>"]
      tokens_to_decode[j] = tokens
      temp_ids = tokenizer.convert_tokens_to_ids(tokens)
      #temp_ids = tokenizer(sen)['input_ids']
      ids_to_decode[j] = temp_ids
    

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    it = iter(loader)
      

    with torch.no_grad():
        for batch in tqdm(it):
            batch_attn_probs = []  # To store attention weights for each layer in the current batch
            torch.cuda.empty_cache()
            input_ids = batch.to(device)
            outputs = model(input_ids=input_ids, output_attentions=True, decoder_input_ids=input_ids)
            attn = outputs.cross_attentions
            for j in range(len(attn[layer])):
              attention_weights[idx * batch_size + j] = (attn[layer][j][head][0]).to('cpu')
            idx+=1

    return attention_weights, dataset, tokens_to_decode




# def run_attn_examples(input_sentences, layer, head, batch_size=128):
#     """
#     Returns Attention weights for selected Layer and Head along with ids and tokens
#     of the input_sentence
#     """
#     ids = []
#     ids_to_decode = [None for k in range(len(input_sentences))]
#     tokens_to_decode = [None for k in range(len(input_sentences))]
#     segment_ids = []
#     input_masks = []
#     attention_weights = [None for z in input_sentences]
#     ## BERT pre-processing
#     for j,sen in enumerate(tqdm(input_sentences)):
        
#         text_tokens = tokenizer(sen)['input_ids']
#         if len(text_tokens) >= max_seq_len-2:
#             text_tokens = text_tokens[:max_seq_len-4]
#         tokens = text_tokens + ["</s>"]
#         tokens_to_decode[j] = tokens
#         temp_ids = tokenizer.convert_tokens_to_ids(tokens)
#         ids_to_decode[j] = temp_ids
#         input_mask = [1] * len(temp_ids)
#         segment_id = [0] * len(temp_ids)
#         padding = [0] * (max_seq_len - len(temp_ids))
        
        
#         temp_ids += padding
#         input_mask += padding
#         segment_id += padding
        
#         ids.append(temp_ids)
#         input_masks.append(input_mask)
#         segment_ids.append(segment_id)
    
#     # Convert Ids to Torch Tensors
#     ids = torch.tensor(ids) 
#     segment_ids = torch.tensor(segment_ids)
#     input_masks = torch.tensor(input_masks)
    
#     steps = len(ids) // batch_size
#     idx = 0
#     for i in trange(steps+1):
#         if i == steps:
#             temp_ids = ids[i * batch_size : len(ids)]
#             temp_segment_ids = segment_ids[i * batch_size: len(ids)]
#             temp_input_masks = input_masks[i * batch_size: len(ids)]
#         else:
#             temp_ids = ids[i * batch_size : i * batch_size + batch_size]
#             temp_segment_ids = segment_ids[i * batch_size: i * batch_size + batch_size]
#             temp_input_masks = input_masks[i * batch_size: i * batch_size + batch_size]
        
#         temp_ids = temp_ids.to(device)
#         temp_segment_ids = temp_segment_ids.to(device)
#         temp_input_masks = temp_input_masks.to(device)
#         with torch.no_grad():
#              outputs = model(temp_ids, temp_segment_ids, temp_input_masks, output_attentions=True)
#              attn = outputs.cross_attentions
             
             
#         for j in range(len(attn[layer])):
#             #print(len(attn[layer][j]))
#             attention_weights[i * batch_size + j] = (attn[layer][j][head][1]).to('cpu')
    
#     return attention_weights, ids_to_decode, tokens_to_decode

In [ ]:
def prepare_data(aw, ids_to_decode, tokens_to_decode):
    out_sen = [None for i in range(len(aw))]
    for i in tqdm(range(len(aw))): 
        clear_ram_cache()
        #topv, topi = aw[i].topk(len(inps_tokens[i]))
        
        p = ids_to_decode[i].numpy().tolist()
       
        
        topv, topi = aw[i].topk(p.index(0))
        topi = topi.tolist()
        topv = topv.tolist()
        
        #print("Original Top Indexes = {}".format(topi))
        
        topi = [topi[j] for j in range(len(topi)) if ids_to_decode[i][topi[j]] not in not_to_remove_ids] # remove noun and common words
        #print("After removing Nouns = {}".format(topi))
        
        #topi = [topi[j] for j in range(len(topi)) if "|" not in tokens_to_decode[i][topi[j]]]
        #topi = [topi[j] for j in range(len(topi)) if "|" not in tokens_to_decode[i][topi[j]]]
         # Remove half words
        #print("After removing Half-words = {}".format(topi))

        if (len(topi) < 4 and len(topi) > 0):
            topi = [topi[0]]
        elif(len(topi) < 8):
            topi = topi[:2]
        else:
            topi = topi[:3]

        #print("Final Topi = {}".format(topi))
        #print("input_ids = {}".format(ids_to_decode))
        final_indexes = []
        count = 0
        count1 = 0
        #print(ids_to_decode[i], tokens_to_decode[i])
        #Tuple of torch.FloatTensor (one for each layer) of shape (batch_size, num_heads, sequence_length, sequence_length).
        while ids_to_decode[i][count] != 0:
            if count in topi:
                while ids_to_decode[i][count + count1 + 1] != 0:
                    if "|" in tokens_to_decode[i][count + count1 + 1]:
                        count1 += 1
                    else:
                        break
                count += count1
                count1 = 0
            else:
                final_indexes.append(ids_to_decode[i][count])
            count += 1

        #print(final_indexes)
        temp_out_sen = tokenizer.decode(final_indexes,skip_special_token=True)
        #print(temp_out_sen, "\n\n")
        #temp_out_sen = " ".join(temp_out_sen).replace("</s>","").replace("|", "")

        temp_out_sen = temp_out_sen.split("</s>")[0]
        out_sen[i] = temp_out_sen.strip()
    
    return out_sen

In [ ]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(train_0_data, layer=6, head=0, batch_size=128)
train_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(train_0_data, train_0_out_sen, train_0_out, sentiment="<NEG>")

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 177218/177218 [27:09<00:00, 108.79it/s]


/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.train.0


In [ ]:
train_0_out_sen

In [ ]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(train_1_data, layer=6, head=0, batch_size=128)
train_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(train_1_data, train_1_out_sen, train_1_out, sentiment="<POS>")

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 266041/266041 [39:15<00:00, 112.97it/s]


/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.train.1


In [ ]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(dev_0_data, layer=6, head=0, batch_size=128)
dev_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(dev_0_data, dev_0_out_sen, dev_0_out, sentiment="<NEG>")

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 2000/2000 [00:34<00:00, 58.22it/s] 

/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.dev.0


In [ ]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(dev_1_data, layer=6, head=0, batch_size=128)
dev_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(dev_1_data, dev_1_out_sen, dev_1_out, sentiment="<POS>")

100%|██████████| 2000/2000 [00:17<00:00, 112.62it/s]

/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.dev.1


In [ ]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(test_1_data, layer=6, head=0, batch_size=128)
test_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(test_1_data, test_1_out_sen, test_1_out, sentiment="<POS>")

100%|██████████| 500/500 [00:04<00:00, 112.37it/s]

/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.test.1


In [ ]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(test_0_data, layer=6, head=0, batch_size=128)
test_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(test_0_data, test_0_out_sen, test_0_out, sentiment="<NEG>")

100%|██████████| 500/500 [00:05<00:00, 85.00it/s] 

/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.test.0


In [ ]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(ref_1_data, layer=6, head=0, batch_size=128)
ref_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_ref_output_file(ref_1_out_sen, reference_1_out,sentiment="<NEG>")

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 500/500 [00:00<00:00, 690989.13it/s]


In [ ]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(ref_0_data, layer=6, head=0, batch_size=128)
ref_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_ref_output_file(ref_0_out_sen, reference_0_out, sentiment="<POS>")

100%|██████████| 500/500 [00:00<00:00, 673459.22it/s]


In [ ]:
ta = data2 = ""
 
# Reading data from file1
with open('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.train.0') as fp:
    data = fp.read()
 
# Reading data from file2
with open('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.train.1') as fp:
    data2 = fp.read()
 
# Merging 2 files
# To add the data of file2
# from next line
data += "\n"
data += data2
 
with open ('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.train', 'w+') as fp:
    fp.write(data)

In [ ]:
ta = data2 = ""
 
# Reading data from file1
with open('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.dev.0') as fp:
    data = fp.read()
 
# Reading data from file2
with open('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.dev.1') as fp:
    data2 = fp.read()
 
# Merging 2 files
# To add the data of file2
# from next line
data += "\n"
data += data2
 
with open ('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.dev', 'w+') as fp:
    fp.write(data)

In [ ]:
ta = data2 = ""
 
# Reading data from file1
with open('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.test.0') as fp:
    data = fp.read()
 
# Reading data from file2
with open('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.test.1') as fp:
    data2 = fp.read()
 
# Merging 2 files
# To add the data of file2
# from next line
data += "\n"
data += data2
 
with open ('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/sentiment.test', 'w+') as fp:
    fp.write(data)

In [ ]:
ta = data2 = ""
 
# Reading data from file1
with open('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/reference.0') as fp:
    data = fp.read()
 
# Reading data from file2
with open('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/reference.1') as fp:
    data2 = fp.read()
 
# Merging 2 files
# To add the data of file2
# from next line
data += "\n"
data += data2
 
with open ('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/reference', 'w+') as fp:
    fp.write(data)